In [3]:
# Dependencies
# Dependencies
import numpy as np
#import tweepy
import time
import json
#from config import consumer_key, consumer_secret, access_token, access_token_secret
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
#Import Nutrition Facts  from file 
# load CSV
nutr_file = "Nutrition Data for Fast Food 2017.csv"
nutr_pd = pd.read_csv(nutr_file, encoding='iso-8859-1', low_memory=False)
restu_list=nutr_pd.Fast_Food_Restaurant.unique()
print(restu_list)

["McDonald's" 'Burger King' "Wendy's" 'Chick-fil-A' 'Jack in the Box'
 'Sonic' 'Dairy Queen' "Carl's Jr." "Hardee's" 'White Castle'
 'Whataburger' 'In-N-Out Burger']


In [ ]:
#Set header for YelpFusion API
headers = {'Authorization': 'Bearer {0}'.format(api_key)}

#set an empty list to append dictionary of each Fast Food and their rating 
restaurants_rating = []

#Loop through the restaurants to check if it exists in SF
for restaurant in restu_list:
    #Set parameters
    params = {
        "term": restaurant,
        "location": "San Francisco",
        "state": "CA",
        "country": "US",
        "limit": 30
    }

    # Build the endpoint URL (Checks all ice cream shops)
    base_url = "https://api.yelp.com/v3/businesses/search"

    # Run a request to endpoint and convert result to json
    yelp_data = requests.get(base_url, params=params, headers=headers)
    yelp_data = yelp_data.json()
        
    #Loop through all the businesses returned and find which restaurants are in SF    
    for business in yelp_data['businesses']:
        
        #Using the business id, run a request via Yelp's Business Lookup to retrieve the city value
        yelp_data_id = requests.get(f'https://api.yelp.com/v3/businesses/{business["id"]}', headers=headers)
        yelp_data_id = yelp_data_id.json()
        
        #Since the results returned are 'best matches', filter out all the restaurants that have the same name and is located in SF
        if business['name'] in restaurant and yelp_data_id['location']['city'] == "San Francisco":
            
            #Append a dictionary containing the fast food and it's rating
            restaurants_rating.append({"Fast Food": restaurant,
                               "Rating": business['rating']})

pprint(restaurants_rating)

In [ ]:
#Create a dataframe from the dictionary 
restaurant_pd = pd.DataFrame.from_dict(restaurants_rating)

#Group by the Fast Food
restaurant_group = restaurant_pd.groupby("Fast Food")

#Get the average rating for each restaurant
average_rating = restaurant_group["Rating"].mean()

#Get the number of restaurants
num_restaurants = restaurant_group["Fast Food"].count()

#Create a new dataframe to hold the average rating and number of restaurants
restaurant_df = pd.DataFrame({"Average Rating": average_rating.map("{:,.2f}".format),
                   "Number of Restaurants": num_restaurants})

#Sort by Average Rating descending
restaurant_df = restaurant_df.sort_values("Average Rating", ascending=0)

#Retrieve the columns in the order we want 
restaurant_df = restaurant_df[["Average Rating","Number of Restaurants"]]

#Print out the final dataframe
restaurant_df

In [ ]:
# Yelp api 